In [2]:
import geopandas as gpd
import pandas as pd
import os
import parcelfunks
import pyogrio
import numpy as np
import importlib

In [2]:
countiesPath = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
adamsPath = os.path.join(countiesPath,'08001')

In [72]:
parcels = gpd.read_file(os.path.join(adamsPath,'parcels.shp'))
parcels.to_crs(crs='ESRI:102003',inplace=True)

In [52]:
buildings = gpd.read_file(os.path.join(adamsPath,'08001_buildings.shp'))
buildings.to_crs(crs='ESRI:102003', inplace=True)


In [73]:
parcels = parcelfunks.parcelBuildings(parcels,buildings)


In [74]:
len(parcels)

104143

In [75]:
parcels = parcelfunks.parcelPreFilter(parcels)

In [76]:
len(parcels)

303

In [87]:
parcels['UNIQID'] = np.random.randint(low=1, high=1000000000, size=len(parcels))

In [89]:
costarParcels = gpd.read_file(os.path.join(adamsPath,'08001.gpkg'),layer='MH_parcels')

In [90]:
costarParcels['COSTAR'] = 1

In [91]:
parcels2 = parcels.merge(costarParcels[['COSTAR','UNIQID']], on='UNIQID')

KeyError: "['UNIQID'] not in index"

In [34]:
parcels2.loc[parcels2['COSTAR']==1]

,APN,APN2,STATE,COUNTY,FIPS,SIT_HSE_NU,SIT_DIR,SIT_STR_NA,SIT_STR_SF,SIT_FULL_S,...,MINX,MINY,MAXX,MAXY,VERSION,QUANTARIUM,ATTDATE,geomCalc,geometry,COSTAR


In [64]:
parcels2

,APN,APN2,OWNER,geometry,Sum_Within,mnBlgArea,intLen,intZscore,extLen1,extZscore1,COSTAR
0,171921200006,70291,AMC REDWOOD ESTATES LLC,"MULTIPOLYGON (((-762763.731 301698.449, -76278...",670.0,121.26129,0,0.02231,48,8.557194,1


array([553482160, 739325107, 984633133, 191574206, 815452172, 780708074,
       159302158,  54130428, 436011103, 910628661, 189773439, 344894028,
       137049261, 966207275, 854741692, 550508391, 280253576, 429953311,
       770389110, 988129159, 910479408, 704067504, 845778159, 282961557,
       932902912, 469288882, 276863075, 947999552, 279549275, 553900066,
       132278179,  16039301, 795393389, 726576165, 114691093,  21121414,
       456065228, 883429572, 237224637,  79192326, 685004882, 983347935,
       580126957, 864041240, 324885988, 468561362, 154948682, 914101947,
       105138946, 177389759, 113309341, 726382926, 558824558, 626141047,
       640752203, 840062045, 617798843, 556459436, 585082750, 142009181,
       877463646, 529697861, 579011229, 507003058, 449344345, 426232162,
         7724620,  43025874, 265453360, 410751370,   6248685, 849073615,
       312860788, 700811349, 471317018, 793811925, 923354508, 256348938,
       201074455, 621384696, 247930957, 499122074, 

renaming original mhp data:

In [25]:
state = '08'
CO_path = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties'
#CO_pInventory = pInventory.loc[pInventory['STATE']=='08']
CO_pInventory = pd.read_csv(r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\counties\ColoradoInventory.csv', dtype={'STATE':str,'COUNTY':str})
CO_pInventory_True = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==True]
CO_pInventory_False = CO_pInventory.loc[CO_pInventory['DATA_PRESENT']==False]
CO_pInventory_False.to_csv(os.path.join(CO_path,'missingParcelData.csv')) # need to get this to work
fipsList = CO_pInventory_True['COUNTY'].tolist()   
parcelsPaths = [os.path.join(CO_path,fips) for fips in fipsList]

In [101]:
coMHPPath = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\08_MHPs_OG.gpkg'

In [100]:
pyogrio.list_layers(coMHPs)

array([['08_mhps_non_matched', 'Point'],
       ['08_mhps_non_matched_multi100_less5', 'Point'],
       ['08_missed_buffs', 'Polygon'],
       ['08_MHPs_OG', 'Point'],
       ['08_MHPs_Prepped_OG', 'Point']], dtype=object)

In [102]:
coMHPs = gpd.read_file(coMHPPath,layer='08_MHPs_OG')

In [105]:
coMHPs.COUNTYFIPS

0      08123
1      08013
2      08069
3      08101
4      08107
       ...  
679    08083
680    08083
681    08083
682    08105
683    08105
Name: COUNTYFIPS, Length: 684, dtype: object

In [111]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    countyMHPs = coMHPs.loc[coMHPs['COUNTYFIPS']==fips]
    countyMHPs.to_csv(os.path.join(path,f'MHP_{fips}.csv'))

BLOW UP OLD ONES

In [106]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    mhpPath = os.path.join(path,f'MHP_{fips}.csv')
    if os.path.exists(mhpPath):
        os.remove(mhpPath)

In [109]:
path

'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Counties\\08001'

08123 problem:

In [46]:
weldPath = os.path.join(countiesPath,'08123')
adamsPath = os.path.join(countiesPath,'08001')
oteroPath = os.path.join(countiesPath,'08089')

In [47]:
import importlib

### trouble shooting tempID problem in parcelMHPJoin

In [141]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [ ]:
parcelfunks.parcelWorker(adamsPath)

In [179]:
adams = gpd.read_file(os.path.join(adamsPath,'08001.gpkg'))
parcels = gpd.read_file(os.path.join(adamsPath,'parcels.shp'))
parcels.to_crs(crs='ESRI:102003', inplace=True)

In [143]:
buildings = gpd.read_file(os.path.join(adamsPath,'08001_buildings.shp'))
buildings.to_crs(crs='ESRI:102003', inplace=True)

In [180]:
parcels = parcelfunks.parcelBuildings(parcels,buildings)
parcels = parcelfunks.parcelPreFilter(parcels)

In [181]:
parcels['UNIQID'] = np.random.randint(low=1, high=1000000000, size=len(parcels))

In [182]:
costarHomes = gpd.read_file(os.path.join(adamsPath,'08001_COSTAR_mhps.gpkg'),layer='COSTAR_mhps')
costarHomes.to_crs(crs='ESRI:102003', inplace=True)

In [183]:
costarParcels = parcelfunks.parcelCostarJoin(parcels,costarHomes)

In [185]:
costarParcels['COSTAR']=1

In [ ]:
costarParcels

In [187]:
parcels_postCostar = parcels.merge(costarParcels[['COSTAR', 'UNIQID']], how='left', on='UNIQID')


In [189]:
1 in parcels_postCostar['COSTAR']

True

In [202]:
remainingParcels = parcels_postCostar.loc[parcels_postCostar['COSTAR']!=1]

In [193]:
len(parcels), len(remainingParcels)

(303, 281)

In [194]:
mhps = gpd.read_file(os.path.join(adamsPath,'08001_mhps_OG.gpkg'), layer='08001_MHPS_OG_prepped')
mhps.to_crs(crs='ESRI:102003', inplace=True)

In [203]:
cols = remainingParcels.columns

In [215]:
cols = cols.drop('tempID')

In [216]:
cols

Index(['APN', 'APN2', 'OWNER', 'geometry', 'Sum_Within', 'mnBlgArea', 'intLen',
       'intZscore', 'extLen1', 'extZscore1', 'UNIQID', 'IDcheck', 'COSTAR'],
      dtype='object')

In [208]:
remainingParcels['tempID'] = remainingParcels.index

In [209]:
mhps_parcels_near = gpd.sjoin_nearest(mhps,remainingParcels,max_distance=50, distance_col='distances')

In [197]:
parcels.index

RangeIndex(start=0, stop=303, step=1)

In [175]:
apnParcel = parcelfunks.apnJoin(parcels,costarHomes)

In [164]:
cols = apnParcel.columns

In [176]:
parcels['tempID'] = parcels.index

In [177]:
mhps_parcels_near = gpd.sjoin_nearest(costarHomes,parcels,max_distance=50, distance_col='distances')


In [178]:
mhps_parcels_near.columns

Index(['MH_prop_id', 'MH_prop_name', 'MH_prop_city', 'MH_prop_state',
       'MH_prop_county', 'MH_prop_zip', 'MH_lat', 'MH_long', 'MH_APN',
       'MH_units', 'MH_COUNTY_FIPS', 'geometry', 'index_right', 'APN', 'APN2',
       'OWNER', 'Sum_Within', 'mnBlgArea', 'intLen', 'intZscore', 'extLen1',
       'extZscore1', 'UNIQID', 'tempID', 'distances'],
      dtype='object')

In [148]:
costarParcels['COSTAR']=1

In [149]:
parcelsMerged = parcels.merge(costarParcels[['COSTAR', 'UNIQID']], how='left', on='UNIQID')

In [150]:
len(parcelsMerged)

303

In [151]:
remainingParcels = parcels.loc[parcelsMerged['COSTAR']!=1]

In [152]:
len(remainingParcels)

281

In [27]:
parcelfunks.unionWorker(adamsPath)

In [45]:
parcelfunks.mergeWorker(adamsPath)

Now fixing/refactoring output concat stuff

In [218]:
outDir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'

Blow up old ones:

In [219]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    finalPath = os.path.join(path,f'{fips}.gpkg')
    if os.path.exists(finalPath):
        os.remove(finalPath)

In [253]:
# blow up:
os.remove(os.path.join(outDir,'Colorado_Final.gpkg'))

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\Colorado_Final.gpkg'

In [227]:
import pyogrio

In [230]:
pyogrio.list_layers(os.path.join(adamsPath,'08001.gpkg'))

array([['COSTAR_parcels', 'MultiPolygon'],
       ['MHP_parcels', 'MultiPolygon'],
       ['COSTAR_parc_blk_union2000', 'MultiPolygon'],
       ['MHP_parc_blk_union2000', 'MultiPolygon'],
       ['COSTAR_parc_blk_union2010', 'MultiPolygon'],
       ['MHP_parc_blk_union2010', 'MultiPolygon']], dtype=object)

In [231]:
for version in ['COSTAR','MHP']:
    print(version)

COSTAR
MHP


GOOD!:

In [288]:
versions = {'COSTAR':{'_parcels','_parc_blk_union2000','_parc_blk_union2010'}, 'MHP':{'_parcels','_parc_blk_union2000','_parc_blk_union2010'}}

In [254]:
versions = {'COSTAR':{'_parcels','_parc_blk_union2000','_parc_blk_union2010'}, 'MHP':{'_parcels','_parc_blk_union2000','_parc_blk_union2010'}}
for key, values in versions.items():
    for v in values:
        outDF = pd.DataFrame()
        for path in parcelsPaths:
            fips = path.split('\\')[-1]        
            if os.path.exists(os.path.join(path,fips+'.gpkg')):
                layers = pyogrio.list_layers(os.path.join(path,fips+'.gpkg'))
                if f'{key}{v}' in layers:
                    outDF = pd.concat([outDF,gpd.read_file(os.path.join(path, fips+'.gpkg'),layer=f'{key}{v}')])
        outDF.to_file(os.path.join(outDir,'Colorado_Final.gpkg'),layer=f'{key}{v}')

In [277]:
stateDir = rf'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\CO_2022' #change later to {state}
outDir = rf'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\{state}_output'

In [278]:
stateFinalPath = os.path.join(outDir,f'{state}_Final.gpkg')
for keys, values in versions.items():
    for key in keys:
        for value in values:
            mhps = pd.read_csv(os.path.join(stateDir,f'{version}_mhps.csv'), dtype=)
            layer = gpd.read_file(stateFinalPath, layer=f'{key}{v}')
            outputDF = pd.concat(outputDF,layer)

            
    

TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

In [255]:
for k,v in versions.items():
    print(k)

COSTAR
MHP


In [34]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

Now csv outputs and exceptions:

In [ ]:
#final county unions concat to state level and output:
state = '08'
outDir = rf'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\{state}_output'
versions = {'COSTAR':{'_parcels',
                      '_parc_blk_union2000',
                      '_parc_blk_union2010'}, 
               'MHP':{'_parcels',
                      '_parc_blk_union2000',
                      '_parc_blk_union2010'}}
years = ['2000','2010']

exceptionsFinalDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    countyEx = pd.read_csv(os.path.join(path,'exceptions.csv'), dtype={'COUNTY_FIPS':str})
    exceptionsFinalDF = pd.concat([exceptionsFinalDF,countyEx])
    for version in versions.keys():
        outDF = pd.DataFrame()
        for year in years:
            yr = year[-2:]                  
            dtype={f'STATEFP{yr}':str,f'COUNTYFP{yr}':str,f'TRACTCE{yr}':str,f'BLOCKCE{yr}':str,f'GEOID{yr}':str,f'MTFCC{yr}':str,f'UACE{yr}':str,f'GEOID{yr}':str, 'MH_COUNTY_FIPS':str}
            if version == 'COSTAR':
                dtype.update({'MH_APN':str, 'APN':str, 'APN2':str})            
            filePath = os.path.join(path,f'{version}_{fips}_{year}.csv')            
            if os.path.exists(filePath):
                unionDF = pd.read_csv(filePath,dtype=dtype)
                print(len(unionDF))
                outDF = pd.concat([outDF,unionDF])
                print(len(outDF))
            else:
                print(filePath, 'does not exist')
        outDF.to_csv(os.path.join(outDir, f'{state}_{version}_{year}_final.csv'))
exceptionsFinalDF.to_csv(os.path.join(outDir, f'{state}_exceptions.csv'))



In [45]:
#NEW
state = '08'
outDir = rf'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\{state}_output'
versions = {'COSTAR':{'_parcels',
                      '_parc_blk_union2000',
                      '_parc_blk_union2010'}, 
               'MHP':{'_parcels',
                      '_parc_blk_union2000',
                      '_parc_blk_union2010'}}
years = ['2000','2010']

for version in versions.keys():    
    for year in years:
        yr = year[-2:]
        outDF = pd.DataFrame()      
        for path in parcelsPaths:
            fips = path.split('\\')[-1]                
            dtype={f'STATEFP{yr}':str,f'COUNTYFP{yr}':str,f'TRACTCE{yr}':str,f'BLOCKCE{yr}':str,f'GEOID{yr}':str,f'MTFCC{yr}':str,f'UACE{yr}':str,f'GEOID{yr}':str, 'MH_COUNTY_FIPS':str}
            if version == 'COSTAR':
                dtype.update({'MH_APN':str, 'APN':str, 'APN2':str})            
            filePath = os.path.join(path,f'{version}_{fips}_{year}.csv')            
            if os.path.exists(filePath):
                unionDF = pd.read_csv(filePath,dtype=dtype)
                outDF = pd.concat([outDF,unionDF])
        outDF.drop(outDF.filter(regex='Unnamed*').columns,axis=1, inplace=True)
        outDF.to_csv(os.path.join(outDir, f'{state}_{version}_{year}_final.csv'))


exceptionsFinalDF = pd.DataFrame()
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    countyEx = pd.read_csv(os.path.join(path,'exceptions.csv'), dtype={'COUNTY_FIPS':str})
    exceptionsFinalDF = pd.concat([exceptionsFinalDF,countyEx])
exceptionsFinalDF.to_csv(os.path.join(outDir, f'{state}_exceptions.csv'))

In [41]:
f'{version}_{fips}_{year}.csv'

"('MHP', {'_parc_blk_union2000', '_parc_blk_union2010', '_parcels'})_08125_2010.csv"

In [260]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    if os.path.exists(os.path.join(path,f'MHP_{fips}_COSTAR.csv')):
        costar = pd.read_csv(os.path.join(path,f'MHP_{fips}_COSTAR.csv'), dtype={'MH_COUNTY_FIPS':str})
        costar.to_csv(os.path.join(path,f'COSTAR_{fips}.csv'))

In [262]:
yr='10'
dtype={f'STATEFP{yr}':str,f'COUNTYFP{yr}':str,f'TRACTCE{yr}':str,f'BLOCKCE{yr}':str,f'GEOID{yr}':str,f'MTFCC{yr}':str,f'UACE{yr}':str,f'GEOID{yr}':str, 'MH_COUNTY_FIPS':str}

In [265]:
dtype.update({'MH_APN':str, 'APN':str, 'APN2':str})

In [266]:
dtype

{'STATEFP10': str,
 'COUNTYFP10': str,
 'TRACTCE10': str,
 'BLOCKCE10': str,
 'GEOID10': str,
 'MTFCC10': str,
 'UACE10': str,
 'MH_COUNTY_FIPS': str,
 'MH_APN': str,
 'APN': str,
 'APN2': str}

In [272]:
'2020'[-2:]

'20'

In [279]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    costarPath = os.path.join(path,f'{fips}_COSTAR_mhps.gpkg')
    if os.path.exists(os.path.join(path,f'{fips}_COSTAR_mhps.gpkg')):
        mhps = gpd.read_file(costarPath,layer='COSTAR_mhps')
        mhps.to_csv(os.path.join(path, 'COSTAR_mhps.csv'))

In [284]:
costar = gpd.read_file(os.path.join(stateDir,'COSTAR_mhps.gpkg'),layer='COSTAR_mhps')
costar.to_csv(os.path.join(stateDir,'COSTAR_mhps.csv'))

In [10]:
stateDir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'

Debug mhp_union_merge:

In [33]:
importlib.reload(parcelfunks)

adamsDir = os.path.join(stateDir,'counties','08001')
fips = '08001'
costarName = f'COSTAR_{fips}.csv'
mhpName = f'MHP_{fips}.csv'
costarPath = os.path.join(adamsDir,costarName)

In [12]:
costarPath

'C:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\counties\\08001\\COSTAR_08001.csv'

In [24]:
parcelfunks.mhp_union_merge('08001', 'MHP', adamsDir, mhpName)

In [17]:
os.path.exists(costarPath)

True

In [31]:
for path in parcelsPaths:
    fips = path.split('\\')[-1]
    years = ['2000','2010']
    for year in years:
        file = os.path.join(path,f'MHP_union_csv{year}.csv')
        if os.path.exists(file):
            os.remove(file)

Why is parcel APN in numeric? 

In [63]:
importlib.reload(parcelfunks)

<module 'parcelfunks' from 'c:\\Users\\phwh9568\\Data\\ParcelAtlas\\CO_2022\\parcelfunks.py'>

In [47]:
adams = gpd.read_file(os.path.join(adamsDir,'parcels.shp'))

In [64]:
boulderPath = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022\Counties\08013'

In [66]:
bc = gpd.read_file(os.path.join(boulderPath,'08013.gpkg'), layer='COSTAR_parc_blk_union2010')

In [67]:
bc['APN']

0     157508200003
1     146334100061
2     157502303002
3     131515101001
4     131515101001
          ...     
61    157721000024
62    157508200003
63    131509092001
64    146535412001
65    157502303002
Name: APN, Length: 66, dtype: object